In [0]:
from pyspark.sql import functions as F
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
from pyspark.sql import functions as F
import matplotlib.pyplot as plt
import pandas as pd

def plot_engagement_kpis_with_quirks(df):
    """
    Calculate and plot DAU, WAU, MAU trends from a PySpark DataFrame,
    and highlight possible data quirks (seasonality, sparsity, spikes).
    
    DataFrame must have:
        - event_date (date or string)
        - count_views (numeric)
    """

    df = df.withColumn("event_date", F.to_date("event_date"))

    dau_df = (df.groupBy("event_date")
                .agg(F.sum("count_views").alias("DAU_views"))
                .orderBy("event_date"))

    df_week = df.withColumn("week", F.date_trunc("week", "event_date"))
    wau_df = (df_week.groupBy("week")
                .agg(F.sum("count_views").alias("WAU_views"))
                .orderBy("week"))

    df_month = df.withColumn("month", F.date_trunc("month", "event_date"))
    mau_df = (df_month.groupBy("month")
                .agg(F.sum("count_views").alias("MAU_views"))
                .orderBy("month"))

    dau_pd = dau_df.toPandas()
    wau_pd = wau_df.toPandas()
    mau_pd = mau_df.toPandas()


    dau_pd["event_date"] = pd.to_datetime(dau_pd["event_date"])
    wau_pd["week"] = pd.to_datetime(wau_pd["week"])
    mau_pd["month"] = pd.to_datetime(mau_pd["month"])


    plt.figure(figsize=(10,5))
    plt.plot(dau_pd["event_date"], dau_pd["DAU_views"], marker='o', label="DAU")
    plt.title("Daily Active Views Trend")
    plt.xlabel("Date")
    plt.ylabel("Views")
    plt.legend()
    plt.show()

    # --- Detect quirks ---
    quirks = {}

    dau_pd['event_date'] = pd.to_datetime(dau_pd['event_date'])

    ###################################
    # 1. Sparsity: days with zero views
    ###################################

    # Define the full date range from the min to the max date in your data
    min_date = dau_pd['event_date'].min()
    max_date = dau_pd['event_date'].max()
    full_dates = pd.date_range(start=min_date, end=max_date)

    # Days with zero views - guardamos el DataFrame para plot
    zero_days_df = dau_pd[dau_pd["DAU_views"] == 0]

    # Days missing from the dataset
    missing_days = full_dates.difference(dau_pd['event_date'])

    quirks["sparsity_zero_views"] = zero_days_df["event_date"].tolist()
    quirks["sparsity_missing_days"] = missing_days.tolist()

    print("Days with zero views:", quirks["sparsity_zero_views"])
    print("Days missing from dataset:", quirks["sparsity_missing_days"])

    #############################################
    # 2. Spikes: values greater than mean + 3*std
    #############################################

    threshold = dau_pd["DAU_views"].mean() + 3 * dau_pd["DAU_views"].std()
    spikes_df = dau_pd[dau_pd["DAU_views"] > threshold]
    quirks["spikes"] = spikes_df["event_date"].tolist()

    #####################
    # 3. Seasonality hint
    #####################

    weekly_avg = dau_pd.groupby(dau_pd["event_date"].dt.dayofweek)["DAU_views"].mean()
    quirks["seasonality"] = weekly_avg.to_dict()

    # --- Plot ---
    plt.figure(figsize=(14,6))
    plt.plot(dau_pd["event_date"], dau_pd["DAU_views"], marker='o', label="DAU")
    plt.plot(wau_pd["week"], wau_pd["WAU_views"], marker='s', label="WAU")
    plt.plot(mau_pd["month"], mau_pd["MAU_views"], marker='^', label="MAU")

    # Mark zero days
    plt.scatter(zero_days_df["event_date"], zero_days_df["DAU_views"], color='red', label="Zero days", zorder=5)

    # Mark spikes
    plt.scatter(spikes_df["event_date"], spikes_df["DAU_views"], color='orange', label="Spikes", zorder=5)

    plt.title("Engagement KPIs with Data Quirks Highlighted")
    plt.xlabel("Date")
    plt.ylabel("Views")
    plt.legend()
    plt.grid(True)
    plt.show()

    # --- Print quirks summary ---
    print("\n=== Data Quirks Detected ===")
    print(f"1. Sparsity (zero days): {quirks['sparsity_zero_views']}")
    print(f"   Missing days: {quirks['sparsity_missing_days']}")
    print(f"2. Spikes: {quirks['spikes']}")
    print("3. Seasonality pattern (avg views by weekday):")
    for day, avg in quirks["seasonality"].items():
        print(f"   Day {day} (0=Mon): {avg:.2f}")




In [0]:
df = spark.read.table("projectviews.gold.projectviews_daily_summary")
plot_engagement_kpis_with_quirks(df)

In [0]:
def plot_avg_views_by_weekday(df, date_col="event_date", views_col="count_views"):
    """
    Calculate average views by weekday and plot the result.

    Parameters:
    - df: Spark DataFrame with at least two columns: a date column and a views count column.
    - date_col: Name of the date column in df (default "event_date").
    - views_col: Name of the views count column in df (default "count_views").
    """

    # Add a weekday number column (Monday=0, Sunday=6)
    df_with_weekday = df.withColumn(
        "weekday",
        ((F.dayofweek(F.col(date_col)) + 5) % 7)
    ).withColumn(
        "weekday_name",
        F.when(F.col("weekday") == 0, "Monday")
         .when(F.col("weekday") == 1, "Tuesday")
         .when(F.col("weekday") == 2, "Wednesday")
         .when(F.col("weekday") == 3, "Thursday")
         .when(F.col("weekday") == 4, "Friday")
         .when(F.col("weekday") == 5, "Saturday")
         .when(F.col("weekday") == 6, "Sunday")
    )

    # Aggregate average views by weekday name
    agg_df = df_with_weekday.groupBy("weekday_name") \
        .agg(F.avg(views_col).alias("avg_views"))

    # Collect to pandas
    pdf = agg_df.toPandas()

    # Order the days of the week properly
    order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    pdf['weekday_name'] = pd.Categorical(pdf['weekday_name'], categories=order, ordered=True)
    pdf = pdf.sort_values('weekday_name')

    # Plotting
    plt.figure(figsize=(10,6))
    plt.bar(pdf['weekday_name'], pdf['avg_views'], color='skyblue')
    plt.title('Average Views by Weekday')
    plt.xlabel('Weekday')
    plt.ylabel('Average Views')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

plot_avg_views_by_weekday(df)
